In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import keras

In [2]:
#데이터 다운로드
df = pd.read_excel(r'C:\Users\Hon Kim\Desktop\관광통계\2019년_국민여행조사_원자료\dfr.xlsx', index = True) #불필요한 열 삭제
df.drop(['ID'],axis=1,inplace=True)
df.drop(['select_reason_2'],axis=1,inplace=True)
df.drop(['select_reason_3'],axis=1,inplace=True)


#일부 데이터 정수형변환
df['JEJU']=df['JEJU'].astype(int)
df['KYUNG_S']=df['KYUNG_S'].astype(int)
df['KYUNG_N']=df['KYUNG_N'].astype(int)
df['JEON_S']=df['JEON_S'].astype(int)
df['JEON_N']=df['JEON_N'].astype(int)
df['CHOONG_S']=df['CHOONG_S'].astype(int)
df['CHOONG_N']=df['CHOONG_N'].astype(int)
df['GANGWON']=df['GANGWON'].astype(int)
df['KYUNGGI']=df['KYUNGGI'].astype(int)
df['SEJONG']=df['SEJONG'].astype(int)
df['ULSAN']=df['ULSAN'].astype(int)
df['DAEJEON']=df['DAEJEON'].astype(int)
df['GWANGJU']=df['GWANGJU'].astype(int)
df['INCHEON']=df['INCHEON'].astype(int)
df['DAEGU']=df['DAEGU'].astype(int)
df['BUSAN']=df['BUSAN'].astype(int)
df['SEOUL']=df['SEOUL'].astype(int)

#데이터 구성요소확인
df['pre_food'].unique()

#결측치 처리
df=df.fillna(0)

#결측치 처리 확인(multi-hot encoding)
df['pre_food'].unique()


array([0., 2.])

In [3]:
#만족도, 재방문률, 추천정도 변수를 합쳐
#타겟변수설정
point = df['Satify']+df['revisit']+df['recommend']

#point2 = point >= 15
#(point2.values==True).sum()

#point2 = point >= 14
#(point2.values==True).sum()

#point2 = point >= 13
#(point2.values==True).sum()

point2 = point >= 12
(point2.values==True).sum()

#point2 = point >= 11
#(point2.values==True).sum()

#point2 = point >= 10
#(point2.values==True).sum()

7852

In [4]:
#데이터 프레임 상 타겟변수 제거
df.drop(['Satify'],axis=1,inplace=True)
df.drop(['revisit'],axis=1,inplace=True)
df.drop(['recommend'],axis=1,inplace=True)
df.head()

,visit_count,visit_acq,stay,COST,NUM,per_COST,cost_ski_watering,cost_camping,cost_hiking,cost_fishing,...,earn_self_classify_7,BMAR_1,BMAR_2,BMAR_3,BMAR_4,BMAR_5,winter,spring,summer,autumn
0,4,1,1,100000.0,1,100000.0,0.0,0.0,0.0,0.0,...,0,0,1,0,0,0,0,0,1,0
1,3,1,1,250000.0,1,250000.0,0.0,0.0,0.0,0.0,...,0,0,1,0,0,0,0,1,0,0
2,3,1,0,50001.0,3,16667.0,0.0,0.0,0.0,0.0,...,0,1,0,0,0,0,0,1,0,0
3,3,1,1,120000.0,1,120000.0,0.0,0.0,0.0,0.0,...,0,0,1,0,0,0,0,0,0,1
4,2,1,2,350001.0,3,116667.0,0.0,0.0,0.0,0.0,...,0,0,1,0,0,0,0,0,1,0


In [5]:
#훈련용 데이터셋, 시험 데이터셋 설정
X_train, X_test, y_train, y_test = train_test_split(df, point2, random_state=42, stratify = point2)
#훈련용 데이터셋 타겟변수 분포 확인
sum(y_train)/len(y_train)

0.7476196521518345

In [6]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [7]:
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [8]:
# 각 레이어의 가중치와 바이어스를 초기화하는 옵션이 있음
# keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=None)
# keras.initializers.RandomUniform(minval=-0.05, maxval=0.05, seed=None)

clf = Sequential([
    Dense(units=16, kernel_initializer='uniform', input_dim=333, activation='relu'),
    Dense(units=18, kernel_initializer='uniform', activation='relu'),
    Dropout(0.25),
    Dense(20, kernel_initializer='uniform', activation='relu'),
    Dense(24, kernel_initializer='uniform', activation='relu'),
    Dense(1, kernel_initializer='uniform', activation='sigmoid')
])

In [9]:
clf.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 16)                5344      
_________________________________________________________________
dense_1 (Dense)              (None, 18)                306       
_________________________________________________________________
dropout (Dropout)            (None, 18)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 20)                380       
_________________________________________________________________
dense_3 (Dense)              (None, 24)                504       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 25        
Total params: 6,559
Trainable params: 6,559
Non-trainable params: 0
______________________________________________________

In [10]:
clf.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [11]:
clf.fit(X_train, y_train, batch_size=15, epochs=10)

Epoch 1/10
526/526 [==============================] - 1s 1ms/step - loss: 0.4843 - accuracy: 0.7775
Epoch 2/10
526/526 [==============================] - 1s 1ms/step - loss: 0.4180 - accuracy: 0.8667
Epoch 3/10
526/526 [==============================] - 1s 1ms/step - loss: 0.3885 - accuracy: 0.8728
Epoch 4/10
526/526 [==============================] - 1s 1ms/step - loss: 0.3397 - accuracy: 0.8805
Epoch 5/10
526/526 [==============================] - 1s 1ms/step - loss: 0.3106 - accuracy: 0.8842
Epoch 6/10
526/526 [==============================] - 1s 1ms/step - loss: 0.2882 - accuracy: 0.8944
Epoch 7/10
526/526 [==============================] - 1s 1ms/step - loss: 0.2720 - accuracy: 0.8989
Epoch 8/10
526/526 [==============================] - 1s 996us/step - loss: 0.2559 - accuracy: 0.9075
Epoch 9/10
526/526 [==============================] - 1s 1ms/step - loss: 0.2393 - accuracy: 0.9155
Epoch 10/10
526/526 [==============================] - 1s 1ms/step - loss: 0.2277 - accuracy: 0.91

In [12]:
# epoch 10, train-accuracy: 0.921
score = clf.evaluate(X_test, y_test, verbose=0)
print(clf.metrics_names)
print(score)

['loss', 'accuracy']
[0.4985150396823883, 0.8305407166481018]


In [30]:
# epoch 25, train-accuracy: 0.964
score = clf.evaluate(X_test, y_test, verbose=0)
print(clf.metrics_names)
print(score)

['loss', 'accuracy']
[0.7549619078636169, 0.8297791481018066]


In [25]:
# epoch 50, train-accuracy: 0.989
score = clf.evaluate(X_test, y_test, verbose=0)
print(clf.metrics_names)
print(score)

['loss', 'accuracy']
[3.3166873455047607, 0.807311475276947]


In [35]:
# epoch 75, train-accuracy: 0.990
score = clf.evaluate(X_test, y_test, verbose=0)
print(clf.metrics_names)
print(score)

['loss', 'accuracy']
[1.821864366531372, 0.7943640351295471]


In [13]:
# 각 레이어의 가중치와 바이어스를 초기화하는 옵션이 있음
# keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=None)
# keras.initializers.RandomUniform(minval=-0.05, maxval=0.05, seed=None)

clf = Sequential([
    Dense(units=16, kernel_initializer='uniform', input_dim=333, activation='relu'),
    Dense(units=18, kernel_initializer='uniform', activation='relu'),
    Dense(20, kernel_initializer='uniform', activation='relu'),
    Dense(24, kernel_initializer='uniform', activation='relu'),
    Dense(1, kernel_initializer='uniform', activation='sigmoid')
])

In [14]:
clf.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [15]:
clf.fit(X_train, y_train, batch_size=15, epochs=10)

Epoch 1/10
526/526 [==============================] - 0s 943us/step - loss: 0.4887 - accuracy: 0.7729
Epoch 2/10
526/526 [==============================] - 1s 1ms/step - loss: 0.4137 - accuracy: 0.8682
Epoch 3/10
526/526 [==============================] - 1s 991us/step - loss: 0.3867 - accuracy: 0.8757
Epoch 4/10
526/526 [==============================] - 1s 1ms/step - loss: 0.3620 - accuracy: 0.8805
Epoch 5/10
526/526 [==============================] - 1s 1ms/step - loss: 0.3389 - accuracy: 0.8841
Epoch 6/10
526/526 [==============================] - 1s 1ms/step - loss: 0.3157 - accuracy: 0.8931
Epoch 7/10
526/526 [==============================] - 0s 918us/step - loss: 0.2917 - accuracy: 0.9026
Epoch 8/10
526/526 [==============================] - 0s 933us/step - loss: 0.2700 - accuracy: 0.9059
Epoch 9/10
526/526 [==============================] - 0s 877us/step - loss: 0.2446 - accuracy: 0.9147
Epoch 10/10
526/526 [==============================] - 0s 837us/step - loss: 0.2215 - accu

In [16]:
# epoch 10, train-accuracy: 0.921
score = clf.evaluate(X_test, y_test, verbose=0)
print(clf.metrics_names)
print(score)

['loss', 'accuracy']
[0.5662147402763367, 0.8271135091781616]
